In [23]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings("ignore")               # 경고 메시지 무시
# warnings.filterwarnings(action='default')     # 경고 메시지 활성화

plt.rcParams['axes.unicode_minus'] = False      # Glyph 8722 오류 해결을 위한 설정
# plt.rc('font', family='NanumBarunGothic')       # 한글 폰트 설정

plt.rcParams["figure.figsize"] = (12, 9)        # 그림 크기 설정

In [24]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

각 Feature 설명

- default : 신용불량자 여부
- student : 학생 여부
- balance : 개인의 총 자산
- income : 수입


## 파일 불러오기

In [5]:
file_PATH = '/content/drive/MyDrive/공부/글로벌소프트웨어캠퍼스/파이썬/Sci-Kit Learn/자료/st_default.csv'

df = pd.read_csv(file_PATH, encoding='utf-8', engine='python')
df.drop(['Unnamed: 0'], axis=1, inplace=True)
df

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/공부/글로벌소프트웨어캠퍼스/파이썬/Sci-Kit Learn/자료/st_default.csv'

In [6]:
feature = df.loc[:, ['balance', 'income']]
feature

NameError: name 'df' is not defined

In [7]:
label = df.loc[:, ['default', 'student']]
label

NameError: name 'df' is not defined

In [8]:
df.student.value_counts()

NameError: name 'df' is not defined

## EDA

In [9]:
df.head()

NameError: name 'df' is not defined

In [10]:
df.info()

NameError: name 'df' is not defined

In [11]:
df.isnull().sum()

NameError: name 'df' is not defined

In [12]:
df.describe()

NameError: name 'df' is not defined

In [13]:
df.default.value_counts()

NameError: name 'df' is not defined

In [14]:
df.default.value_counts()

NameError: name 'df' is not defined

In [15]:
df.head(10)

NameError: name 'df' is not defined

In [16]:
sns.pairplot(df)

plt.show()

NameError: name 'df' is not defined

pairplot 그림 정리
- 변량이 학생일때 수입(income)이 낮고, 학생이 아닐때 수입이 높다.
- default(신용불량자) feature는 balance(총 자산)와 income에 생각보다 높은 영향을 주지 않는다.

In [17]:
sns.heatmap(data=df.corr(), annot=True)

plt.show()

NameError: name 'df' is not defined

## Object column 처리

In [18]:
df[['default', 'student']]

NameError: name 'df' is not defined

In [19]:
#@title Label Encoding
# 람다식 or 함수 둘 중에 하나 선택해서 사용.

yes_or_no = lambda x: 1 if x == 'Yes' else 0

# def yes_or_no(df):
#     value = ''
#     if df == 'Yes':
#         value = 1
#     else:
#         value = 0
#     return value

df['default'] = df.default.apply(yes_or_no)
df['student'] = df.student.apply(yes_or_no)
df

NameError: name 'df' is not defined

In [20]:
#@title One Hot Encoding

default_one_hot = pd.get_dummies(df[['default']])
student_one_hot = pd.get_dummies(df[['student']])
df = pd.concat([df, default_one_hot, student_one_hot], axis=1)
df.drop(['default', 'student'], axis=1, inplace=True)
df

NameError: name 'df' is not defined

## sci-kit learn 패키지 임포트

In [21]:
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score, recall_score, precision_score, confusion_matrix

In [22]:
x_train, x_test, y_train, y_test = train_test_split(feature, label['student'], test_size = 0.2, random_state=5)

NameError: name 'feature' is not defined

In [ ]:
y_train.value_counts()

In [ ]:
# x_train, x_test, y_train, y_test = train_test_split(feature, label['student'], test_size = 0.2)


# 클래스 불균형 조정
idx_0 = y_train[y_train == 0].sample(2352, random_state=33).index.tolist()
idx_1 = y_train[y_train == 1].sample(2352, random_state=33).index.tolist()
random_idx = idx_0 + idx_1
x_train = x_train.loc[random_idx].reset_index(drop=True)
y_train = y_train.loc[random_idx].reset_index(drop=True)


# 모델링
std_pipe = make_pipeline(
    StandardScaler(),
    RandomForestClassifier(n_jobs=-1, random_state=42, n_estimators=1000, max_depth=7, max_features=0.8)
    # DecisionTreeClassifier()
)


prediction = std_pipe.fit(x_train, y_train).predict(x_test)

## 하이퍼 파라미터 튜닝

In [ ]:
# 좆나 오래걸리니 하지말 것.
parameters = {
    'n_jobs':[-1, 2, 3, 5],
    'max_depth':[3,4,5,6,7,10],
    'min_samples_split':[2,3,5,10],
    'n_estimators':[300, 500, 1000, 1500, 2000],
    'max_depth':[2, 3, 5, 7, 9],
}
grid = GridSearchCV(RandomForestClassifier(), param_grid=parameters, cv=10, refit=True)
grid.fit(x_train, y_train)

## 모델 평가

In [ ]:
scores = cross_val_score(std_pipe, df, label['student'], scoring='accuracy', cv=10)
print(f'교차 검증별 정확도:{np.round(scores, 4)}')
print(f'평균 검증별 정확도:{np.round(np.mean(scores), 4)}')

In [ ]:
y_train.value_counts()

In [ ]:
confusion_matrix(prediction, y_test)

In [ ]:
accuracy_score(prediction, y_test)

In [ ]:
recall_score(prediction, y_test)

In [ ]:
precision_score(prediction, y_test)

In [ ]:
sns.histplot(df['income'])

plt.show()

In [ ]:
# 학생 여부에 따른 총 자산, 수입 분포
sns.histplot(df['balance'])

In [ ]:
one_hot = pd.get_dummies(df[['default', 'student']])
df.drop(['default', 'student'], axis=1, inplace=True)

df = pd.concat([df, one_hot], axis=1)